## Problem Statement

Define scores for referrals who have been giving fruitful refers. Fruitful refers translate to the performance of the referees who joined by referral. If a client has a higher trade activation ratio for his/her refers, his/her score must be higher.


## Features

* Age
* income_per_annum
* occupation
* Z20
* app_status
* E2 channel
* iskarvy
* KRA

========= trade behaviour of referrer =========
* number of logins in last year
* num_trades in last year
* months since last login
* months since last trade

========= trade behaviour of referees =========
* average num_logins per month post account open
* average num_trades per month post account open 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

from datetime import datetime
from datetime import date

In [4]:
df_leads = pd.read_csv("../../../data/CRM_leads_zoho.csv")

In [5]:
df_client =  pd.read_csv("../../../data/dw_mst_client.csv")
df_client = df_client[df_client["data_source"] == "Equity"]

In [6]:
df_leads.head()

,Id,Created_Time,CreatedDate,Owner,Email,Mobile,Lead_Source,Lead_Status,Campaign_Source,Created_By,Full_Name,Lead_Product,Lead_Sub_Product,Lead_Sub_Status,Is_Existing_Customer,Disposition,Client_Code,Registration_Source,PAN_Number,Appsource,OS_Type,Lead_source_Category,Client_activated_time,ActivatedDate,Modified_Time,Last_Assigned_Owner,Assigned_To,TestU,Skype_ID,New_Status,Lead_SubSource,DeletedFlag,Lead_Campaign,Lead_Status_for_Vendor,Reference_Lead,Referrer,Reference_Name,Reference_Number,Reference_Client_Code,Referal_Code
0,2543273000000359966,2017-10-03T08:53:29+05:30,2017-10-03,2543273000029840449,VARAVICTORY1@GMAIL.COM,8328617490,Mobile,Product Sold,NaN,2543273000000120009,Saju.Kallupurayidam,Equity,NaN,Client Activated,--None--,NaN,BSARADA7,Web,NaN,NaN,NaN,NaN,2021-03-15T15:04:00+05:30,2021-03-15,2021-03-15T15:04:11+05:30,C160290,NaN,C60527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2543273000000357062,2017-10-03T08:53:29+05:30,2017-10-03,2543273000003549312,NaN,9327238545,Mobile,Fresh Lead,NaN,2543273000000120009,Saju.Kallupurayidam,Equity,NaN,Pre Registration,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01,2021-03-17T09:13:01+05:30,NaN,NaN,A202286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2543273000000357228,2017-10-03T08:53:29+05:30,2017-10-03,2543273000003549312,NaN,9795267339,Mobile,Fresh Lead,NaN,2543273000000120009,Mukunda.More,Equity,NaN,Pre Registration,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01,2021-03-16T09:13:11+05:30,NaN,NaN,A202286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2543273000000358449,2017-10-03T08:53:29+05:30,2017-10-03,2543273000003549312,J.GOKANI@YAHOO.COM,9824235502,Mobile,Fresh Lead,NaN,2543273000000120009,JAGDISH GOKANI VidhitÂ PawarÂ,Equity,NaN,Lead Registration,NaN,NaN,JAGDIGOK,Web,NaN,NaN,NaN,NaN,NaN,1900-01-01,2021-03-17T09:29:27+05:30,NaN,NaN,A202286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2543273000000361305,2017-10-03T08:53:29+05:30,2017-10-03,2543273000162018050,NaN,9246873117,Mobile,Account Opening in Process,NaN,2543273000000120009,Mukunda.More,Equity,NaN,Pre Registration,--None--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01,2021-03-15T08:00:20+05:30,NaN,NaN,C160213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## referrers analysis

In [7]:
df_leads["Reference_Number"] = df_leads["Reference_Number"].astype(str).str.split(".").str[0] 

In [43]:
df_referrer = pd.concat([df_leads[~df_leads["Reference_Number"].isna()].groupby(by = ["Reference_Number"]).nunique()[["Mobile"]], 
                        df_leads[(~df_leads["Reference_Number"].isna()) & (df_leads["Lead_Status"] == "Product Sold")].groupby(by = ["Reference_Number"]).nunique()[["Mobile"]]], 
                        axis = 1)


In [44]:
df_referrer = df_referrer.reset_index()
df_referrer = df_referrer[df_referrer["Reference_Number"].str.len() == 10]

In [45]:
df_referrer.columns = ["Reference_Number", "refer_count", "activation_count"]
df_referrer["activation_count"] = df_referrer["activation_count"].fillna(0)

In [46]:
df_referrer

,Reference_Number,refer_count,activation_count
10,6000015546,1,0.0
11,6000016229,1,0.0
12,6000027031,1,0.0
13,6000031654,31,23.0
14,6000037524,3,0.0
...,...,...,...
34530,9999888891,1,0.0
34531,9999893553,1,1.0
34532,9999903825,2,2.0
34533,9999920530,1,1.0


In [47]:
df_referrer.rename(columns = {"Mobile": "refer_count"}, inplace = True)
df_client["cm_mobile"] = df_client["cm_mobile"].astype(str).str.split(".").str[0]
df_referrer = pd.merge(df_referrer, df_client, 
                      left_on = "Reference_Number", 
                      right_on = "cm_mobile", how = "inner")[["cm_cd", "cm_mobile", "refer_count", "activation_count", 
                                                             "cm_dob", "IncomePA", "cm_occup", "E2Channel", "IsKarvy", 
                                                             "KRA YN", "Last_MobileLoginDate", "LTD"]] 

In [48]:
df_model = df_referrer.copy()

## adding demographcs to referrers

In [51]:
# ## adding age 
# now = datetime.now()
# df_model["cm_dob"] = df_model["cm_dob"].astype(str).str.split(".").str[0]
# df_model["cm_dob"] = pd.to_datetime(df_model["cm_dob"], errors = "coerce")
# df_model["age"] =  (now -df_model["cm_dob"]).astype('<m8[Y]')
# del df_model["cm_dob"]
# print("age column added!!")


# # adding months since last login
# df_model["Last_MobileLoginDate"] = df_model["Last_MobileLoginDate"].astype(str).str.split(".").str[0]
# df_model["Last_MobileLoginDate"] = pd.to_datetime(df_model["Last_MobileLoginDate"], errors = "coerce")
# df_model["months_since_last_login"] =  (now -df_model["Last_MobileLoginDate"])/np.timedelta64(1, 'M')
# del df_model["Last_MobileLoginDate"]
# df_model["months_since_last_login"] = df_model["months_since_last_login"].fillna(132)
# print("month since last login added")

# adding months since last trade - futures
df_model["LTD"] = df_model["LTD"].astype(str).str.split(".").str[0]
df_model["LTD"] = pd.to_datetime(df_model["LTD"], errors = "coerce")
df_model["months_since_last_futures_trade"] =  (now -df_model["LTD"])/np.timedelta64(1, 'M')
del df_model["LTD"]
df_model["months_since_last_trade"] = df_model["months_since_last_trade"].fillna(132)
print("month since last trade added")

month since last trade added


In [52]:
df_model

,cm_cd,cm_mobile,refer_count,activation_count,IncomePA,cm_occup,E2Channel,IsKarvy,KRA YN,age,months_since_last_login,months_since_last_futures_trade
0,76556800,6000015546,1,0.0,5_10L,P,CAT,0,N,22.0,1.791433,132.000000
1,49282624,6000016229,1,0.0,<=1L,S,CAT,0,Y,22.0,0.838641,1.528594
2,60000270,6000027031,1,0.0,<=1L,P,CAT,0,N,22.0,1.134335,1.068625
3,64606053,6000031654,31,23.0,<=1L,S,CAT,0,N,32.0,0.312963,3.171338
4,86330038,6000037524,3,0.0,<=1L,S,CAT,0,Y,23.0,1.035770,2.645660
...,...,...,...,...,...,...,...,...,...,...,...,...
33381,99998155,9999815503,1,0.0,<=1L,B,CAT,0,N,27.0,132.000000,132.000000
33382,INDERSI5,9999888891,1,0.0,1_5L,NaN,Premia,0,N,36.0,0.312963,0.542947
33383,59684125,9999893553,1,1.0,<=1L,P,CAT,0,N,25.0,0.312963,0.542947
33384,SAMPATIW,9999903825,2,2.0,5_10L,P,CAT,0,N,33.0,0.312963,1.331464


In [54]:
## adding trades and logins in one year

df_logins_one_year = pd.read_csv("../../../data/logins_in_one_year.csv") 
df_model.rename(columns = {"ClientCode": "client_code"}, inplace = True)
df_model = pd.merge(df_model, df_logins_one_year, 
                   left_on = "cm_cd", 
                   right_on = "ClientCode", how = "left")[list(df_model.columns) + ["total_logins_one_year"]]
df_model["total_logins_one_year"] = df_model["total_logins_one_year"].fillna(0)

df_trades_one_year = pd.read_csv("../../../data/trades_in_one_year.csv") 
df_model = pd.merge(df_model, df_trades_one_year, 
                   left_on = "cm_cd", 
                   right_on = "Client_Code", how = "left")[list(df_model.columns) + ["total_trades_one_year"]]
df_model["total_trades_one_year"] = df_model["total_trades_one_year"].fillna(0)

In [55]:
df_model

,cm_cd,cm_mobile,refer_count,activation_count,IncomePA,cm_occup,E2Channel,IsKarvy,KRA YN,age,months_since_last_login,months_since_last_futures_trade,total_logins_one_year,total_trades_one_year
0,76556800,6000015546,1,0.0,5_10L,P,CAT,0,N,22.0,1.791433,132.000000,10.0,0.0
1,49282624,6000016229,1,0.0,<=1L,S,CAT,0,Y,22.0,0.838641,1.528594,35.0,1.0
2,60000270,6000027031,1,0.0,<=1L,P,CAT,0,N,22.0,1.134335,1.068625,4.0,1.0
3,64606053,6000031654,31,23.0,<=1L,S,CAT,0,N,32.0,0.312963,3.171338,208.0,5.0
4,86330038,6000037524,3,0.0,<=1L,S,CAT,0,Y,23.0,1.035770,2.645660,69.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33381,99998155,9999815503,1,0.0,<=1L,B,CAT,0,N,27.0,132.000000,132.000000,0.0,0.0
33382,INDERSI5,9999888891,1,0.0,1_5L,NaN,Premia,0,N,36.0,0.312963,0.542947,1367.0,292.0
33383,59684125,9999893553,1,1.0,<=1L,P,CAT,0,N,25.0,0.312963,0.542947,1635.0,63.0
33384,SAMPATIW,9999903825,2,2.0,5_10L,P,CAT,0,N,33.0,0.312963,1.331464,381.0,41.0


In [56]:
df_model.to_csv("../inter_data/referrer_demo_trade.csv")

## adding referees columns

In [65]:
df_leads["Mobile"] = df_leads["Mobile"].astype(str).str.split(".").str[0] 

In [84]:
df_model

,cm_cd,cm_mobile,refer_count,activation_count,IncomePA,cm_occup,E2Channel,IsKarvy,KRA YN,age,months_since_last_login,months_since_last_futures_trade,total_logins_one_year,total_trades_one_year,referees_avg_logins_in_last_year,referees_avg_trades_in_last_year,num_referees_logging_in,num_referees_trading
0,76556800,6000015546,1,0.0,5_10L,P,CAT,0,N,22.0,1.791433,132.000000,10.0,0.0,NaN,NaN,"Empty DataFrame Columns: [Client_Code, total_l...","Empty DataFrame Columns: [Client_Code, total_l..."
1,49282624,6000016229,1,0.0,<=1L,S,CAT,0,Y,22.0,0.838641,1.528594,35.0,1.0,35.0,1.0,Client_Code total_logins_one_year total_tr...,Client_Code total_logins_one_year total_tr...
2,60000270,6000027031,1,0.0,<=1L,P,CAT,0,N,22.0,1.134335,1.068625,4.0,1.0,4.0,1.0,Client_Code total_logins_one_year total_tr...,Client_Code total_logins_one_year total_tr...
3,64606053,6000031654,31,23.0,<=1L,S,CAT,0,N,32.0,0.312963,3.171338,208.0,5.0,208.0,5.0,Client_Code total_logins_one_year total_tr...,Client_Code total_logins_one_year total_tr...
4,86330038,6000037524,3,0.0,<=1L,S,CAT,0,Y,23.0,1.035770,2.645660,69.0,1.0,69.0,1.0,Client_Code total_logins_one_year total_tr...,Client_Code total_logins_one_year total_tr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33381,99998155,9999815503,1,0.0,<=1L,B,CAT,0,N,27.0,132.000000,132.000000,0.0,0.0,0.0,0.0,0,0
33382,INDERSI5,9999888891,1,0.0,1_5L,NaN,Premia,0,N,36.0,0.312963,0.542947,1367.0,292.0,0.0,0.0,0,0
33383,59684125,9999893553,1,1.0,<=1L,P,CAT,0,N,25.0,0.312963,0.542947,1635.0,63.0,0.0,0.0,0,0
33384,SAMPATIW,9999903825,2,2.0,5_10L,P,CAT,0,N,33.0,0.312963,1.331464,381.0,41.0,0.0,0.0,0,0


In [98]:
df_model["referees_avg_logins_in_last_year"] = [0]*len(df_model)
df_model["referees_avg_trades_in_last_year"] = [0]*len(df_model)
df_model["num_referees_logging_in"] = [0]*len(df_model)
df_model["num_referees_trading"] = [0]*len(df_model)
df_trades_one_year.rename(columns = {"Client_Code" : "client_code"}, inplace = True)

for i, mob in enumerate(df_model["cm_mobile"]):
    df_temp = df_leads[(df_leads["Lead_Status"] == "Product Sold") & (df_leads["Reference_Number"] == mob)][["Client_Code"]]
    df_temp = df_temp[["Client_Code"]]
    
    df_temp = pd.merge(df_temp, df_logins_one_year, 
                   left_on = "Client_Code", 
                   right_on = "ClientCode", how = "left")[list(df_temp.columns) + ["total_logins_one_year"]]
    
    df_temp = pd.merge(df_temp, df_trades_one_year, 
                   left_on = "Client_Code", 
                   right_on = "client_code", how = "left")[list(df_temp.columns) + ["total_trades_one_year"]]
    df_model["referees_avg_logins_in_last_year"].iloc[i] = df_temp["total_logins_one_year"].mean ()
    df_model["referees_avg_trades_in_last_year"].iloc[i] = df_temp["total_trades_one_year"].mean ()
    df_model["num_referees_logging_in"].iloc[i] = len(df_temp[df_temp["total_logins_one_year"] != 0])
    df_model["num_referees_trading"].iloc[i] = len(df_temp[df_temp["total_trades_one_year"] != 0])
    print("current_referrer: {} with {} refers and {} trading.".format(df_model["cm_cd"].iloc[i], 
                                                                      df_model["refer_count"].iloc[i], 
                                                                      df_model["num_referees_trading"].iloc[i]))


current_referrer: 76556800 with 1 refers and 0 trading.
current_referrer: 49282624 with 1 refers and 0 trading.
current_referrer: 60000270 with 1 refers and 0 trading.
current_referrer: 64606053 with 31 refers and 21 trading.
current_referrer: 86330038 with 3 refers and 0 trading.
current_referrer: 53397131 with 1 refers and 0 trading.
current_referrer: 35962427 with 1 refers and 0 trading.
current_referrer: 60000680 with 1 refers and 0 trading.
current_referrer: JOGCHAN1 with 1 refers and 0 trading.
current_referrer: 89236843 with 2 refers and 0 trading.
current_referrer: 44099708 with 1 refers and 0 trading.
current_referrer: 57786859 with 2 refers and 0 trading.
current_referrer: 57713207 with 1 refers and 1 trading.
current_referrer: 22563495 with 5 refers and 4 trading.
current_referrer: 60001569 with 1 refers and 0 trading.
current_referrer: 88828201 with 1 refers and 0 trading.
current_referrer: 31477469 with 1 refers and 0 trading.
current_referrer: 57233132 with 3 refers and 1

current_referrer: 79546258 with 2 refers and 0 trading.
current_referrer: 74060036 with 1 refers and 1 trading.
current_referrer: 90464470 with 3 refers and 2 trading.
current_referrer: 95663155 with 2 refers and 0 trading.
current_referrer: 40667900 with 6 refers and 0 trading.
current_referrer: SANIYARS with 2 refers and 0 trading.
current_referrer: 83915581 with 1 refers and 1 trading.
current_referrer: 22938815 with 3 refers and 0 trading.
current_referrer: 98921962 with 1 refers and 0 trading.
current_referrer: 64151850 with 1 refers and 1 trading.
current_referrer: 24824363 with 1 refers and 1 trading.
current_referrer: 92822837 with 1 refers and 0 trading.
current_referrer: 27953736 with 2 refers and 0 trading.
current_referrer: 62310811 with 3 refers and 0 trading.
current_referrer: 17684272 with 3 refers and 0 trading.
current_referrer: 13016357 with 1 refers and 1 trading.
current_referrer: 71996182 with 5 refers and 0 trading.
current_referrer: TAMSEA18 with 2 refers and 0 t

current_referrer: ASR88507 with 2 refers and 0 trading.
current_referrer: 33763842 with 1 refers and 0 trading.
current_referrer: 47405414 with 1 refers and 0 trading.
current_referrer: NABAA310 with 6 refers and 0 trading.
current_referrer: 60412281 with 2 refers and 0 trading.
current_referrer: 78550163 with 3 refers and 0 trading.
current_referrer: 55080763 with 3 refers and 2 trading.
current_referrer: 19128342 with 1 refers and 0 trading.
current_referrer: FOKIRCHA with 1 refers and 0 trading.
current_referrer: 20912914 with 3 refers and 1 trading.
current_referrer: 60032888 with 4 refers and 0 trading.
current_referrer: 88086361 with 1 refers and 1 trading.
current_referrer: 97169710 with 1 refers and 0 trading.
current_referrer: 84315703 with 1 refers and 1 trading.
current_referrer: 79879398 with 1 refers and 0 trading.
current_referrer: RUHUALO3 with 2 refers and 0 trading.
current_referrer: 43625639 with 2 refers and 0 trading.
current_referrer: 86555487 with 1 refers and 0 t

current_referrer: NARES875 with 7 refers and 2 trading.
current_referrer: TOPPORAJ with 1 refers and 0 trading.
current_referrer: 62014299 with 1 refers and 0 trading.
current_referrer: 88925860 with 1 refers and 1 trading.
current_referrer: 53210757 with 2 refers and 0 trading.
current_referrer: 52874531 with 1 refers and 0 trading.
current_referrer: 35592970 with 1 refers and 0 trading.
current_referrer: 89862539 with 1 refers and 1 trading.
current_referrer: 46057361 with 3 refers and 2 trading.
current_referrer: SOMA2392 with 1 refers and 0 trading.
current_referrer: MSHOAI27 with 4 refers and 0 trading.
current_referrer: 62018066 with 6 refers and 3 trading.
current_referrer: 85527277 with 7 refers and 2 trading.
current_referrer: 79552340 with 2 refers and 0 trading.
current_referrer: RAVIN382 with 6 refers and 0 trading.
current_referrer: 10720881 with 2 refers and 1 trading.
current_referrer: RAJORAO9 with 1 refers and 1 trading.
current_referrer: RAJEGO11 with 2 refers and 0 t

current_referrer: 26342843 with 2 refers and 0 trading.
current_referrer: DMANDAL6 with 1 refers and 0 trading.
current_referrer: 17927271 with 2 refers and 0 trading.
current_referrer: 41937068 with 2 refers and 0 trading.
current_referrer: KAMDEKUM with 2 refers and 0 trading.
current_referrer: 38486093 with 6 refers and 2 trading.
current_referrer: 96668080 with 2 refers and 0 trading.
current_referrer: 22509884 with 1 refers and 0 trading.
current_referrer: MSHAHE25 with 2 refers and 0 trading.
current_referrer: 43224030 with 3 refers and 0 trading.
current_referrer: 78322558 with 2 refers and 1 trading.
current_referrer: 95258809 with 1 refers and 0 trading.
current_referrer: AARSHRAY with 3 refers and 2 trading.
current_referrer: ASR40697 with 6 refers and 1 trading.
current_referrer: 19227721 with 1 refers and 0 trading.
current_referrer: 47611017 with 1 refers and 1 trading.
current_referrer: 46566890 with 1 refers and 0 trading.
current_referrer: 62077437 with 1 refers and 0 t

...

current_referrer: 41606976 with 1 refers and 1 trading.
current_referrer: 45755754 with 4 refers and 3 trading.
current_referrer: 99778752 with 1 refers and 1 trading.
current_referrer: 22860483 with 3 refers and 1 trading.
current_referrer: 26110443 with 1 refers and 0 trading.
current_referrer: 36439639 with 1 refers and 0 trading.
current_referrer: 48139660 with 1 refers and 0 trading.
current_referrer: 82439879 with 3 refers and 0 trading.
current_referrer: 68122424 with 3 refers and 0 trading.
current_referrer: 79181950 with 1 refers and 1 trading.
current_referrer: 75722816 with 1 refers and 0 trading.
current_referrer: 99785352 with 1 refers and 0 trading.
current_referrer: 65314138 with 2 refers and 1 trading.
current_referrer: 17728877 with 2 refers and 1 trading.
current_referrer: 57145086 with 1 refers and 0 trading.
current_referrer: 43323816 with 1 refers and 0 trading.
current_referrer: 60436270 with 11 refers and 8 trading.
current_referrer: 11627005 with 1 refers and 0 

current_referrer: 64142071 with 1 refers and 0 trading.
current_referrer: 49899359 with 5 refers and 1 trading.
current_referrer: 58870664 with 2 refers and 0 trading.
current_referrer: JAINSAN1 with 1 refers and 1 trading.
current_referrer: 77276878 with 1 refers and 0 trading.
current_referrer: ABHIS929 with 1 refers and 0 trading.
current_referrer: 99833456 with 1 refers and 0 trading.
current_referrer: 90716794 with 1 refers and 1 trading.
current_referrer: MANIS547 with 1 refers and 0 trading.
current_referrer: OPRAKA33 with 1 refers and 0 trading.
current_referrer: 54683502 with 1 refers and 0 trading.
current_referrer: 56417346 with 2 refers and 0 trading.
current_referrer: POONA180 with 3 refers and 0 trading.
current_referrer: 62319034 with 1 refers and 0 trading.
current_referrer: 99836129 with 1 refers and 0 trading.
current_referrer: 20921959 with 1 refers and 0 trading.
current_referrer: 53103394 with 1 refers and 0 trading.
current_referrer: 58598182 with 1 refers and 1 t

current_referrer: ONGRPRDP with 4 refers and 2 trading.
current_referrer: 80307463 with 2 refers and 0 trading.
current_referrer: 76444989 with 1 refers and 1 trading.
current_referrer: MALTPARA with 4 refers and 0 trading.
current_referrer: 70768738 with 1 refers and 0 trading.
current_referrer: 14031963 with 1 refers and 1 trading.
current_referrer: KUMANISS with 2 refers and 1 trading.
current_referrer: B1UMESHK with 1 refers and 0 trading.
current_referrer: MUNKUMA3 with 1 refers and 0 trading.
current_referrer: 34232010 with 1 refers and 0 trading.
current_referrer: MEENAK89 with 1 refers and 0 trading.
current_referrer: RAHULY82 with 1 refers and 0 trading.
current_referrer: SERENE83 with 1 refers and 0 trading.
current_referrer: 73932676 with 1 refers and 0 trading.
current_referrer: 14662784 with 4 refers and 0 trading.
current_referrer: SAN18002 with 1 refers and 0 trading.
current_referrer: 41212688 with 1 refers and 0 trading.
current_referrer: 88052998 with 1 refers and 1 t

current_referrer: 40419082 with 1 refers and 1 trading.
current_referrer: 99939669 with 1 refers and 0 trading.
current_referrer: 48078013 with 1 refers and 1 trading.
current_referrer: RUKMNB13 with 3 refers and 0 trading.
current_referrer: 10789123 with 1 refers and 0 trading.
current_referrer: 79059398 with 1 refers and 0 trading.
current_referrer: 99941615 with 1 refers and 0 trading.
current_referrer: 70644202 with 3 refers and 0 trading.
current_referrer: 45893097 with 2 refers and 1 trading.
current_referrer: 32323705 with 3 refers and 0 trading.
current_referrer: KARTH621 with 1 refers and 0 trading.
current_referrer: CRUBESHK with 1 refers and 0 trading.
current_referrer: 99946241 with 1 refers and 1 trading.
current_referrer: 50206200 with 1 refers and 0 trading.
current_referrer: 49611619 with 1 refers and 0 trading.
current_referrer: 53640805 with 2 refers and 0 trading.
current_referrer: 12097675 with 4 refers and 0 trading.
current_referrer: 11387656 with 5 refers and 0 t

current_referrer: SUNIDE96 with 1 refers and 0 trading.
current_referrer: CHPRAVAS with 1 refers and 1 trading.
current_referrer: NITIN23F with 1 refers and 1 trading.
current_referrer: 46843030 with 1 refers and 0 trading.
current_referrer: RAJOTIA2 with 1 refers and 1 trading.
current_referrer: VINOSH88 with 1 refers and 0 trading.
current_referrer: 79180557 with 4 refers and 3 trading.
current_referrer: 21304685 with 2 refers and 0 trading.
current_referrer: DELA0002 with 1 refers and 1 trading.
current_referrer: PRMMAHAN with 3 refers and 1 trading.
current_referrer: 24422322 with 1 refers and 0 trading.
current_referrer: HIMANDUG with 3 refers and 2 trading.
current_referrer: 99998155 with 1 refers and 0 trading.
current_referrer: INDERSI5 with 1 refers and 0 trading.
current_referrer: 59684125 with 1 refers and 1 trading.
current_referrer: SAMPATIW with 2 refers and 2 trading.
current_referrer: 99999205 with 1 refers and 0 trading.


In [99]:
df_model.to_csv("../inter_data/all_features_added.csv")

In [100]:
df_model

,cm_cd,cm_mobile,refer_count,activation_count,IncomePA,cm_occup,E2Channel,IsKarvy,KRA YN,age,months_since_last_login,months_since_last_futures_trade,total_logins_one_year,total_trades_one_year,referees_avg_logins_in_last_year,referees_avg_trades_in_last_year,num_referees_logging_in,num_referees_trading
0,76556800,6000015546,1,0.0,5_10L,P,CAT,0,N,22.0,1.791433,132.000000,10.0,0.0,NaN,NaN,0,0
1,49282624,6000016229,1,0.0,<=1L,S,CAT,0,Y,22.0,0.838641,1.528594,35.0,1.0,NaN,NaN,0,0
2,60000270,6000027031,1,0.0,<=1L,P,CAT,0,N,22.0,1.134335,1.068625,4.0,1.0,NaN,NaN,0,0
3,64606053,6000031654,31,23.0,<=1L,S,CAT,0,N,32.0,0.312963,3.171338,208.0,5.0,3.833333,1.608696,23,21
4,86330038,6000037524,3,0.0,<=1L,S,CAT,0,Y,23.0,1.035770,2.645660,69.0,1.0,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33381,99998155,9999815503,1,0.0,<=1L,B,CAT,0,N,27.0,132.000000,132.000000,0.0,0.0,NaN,NaN,0,0
33382,INDERSI5,9999888891,1,0.0,1_5L,NaN,Premia,0,N,36.0,0.312963,0.542947,1367.0,292.0,NaN,NaN,0,0
33383,59684125,9999893553,1,1.0,<=1L,P,CAT,0,N,25.0,0.312963,0.542947,1635.0,63.0,477.000000,68.000000,1,1
33384,SAMPATIW,9999903825,2,2.0,5_10L,P,CAT,0,N,33.0,0.312963,1.331464,381.0,41.0,45.500000,2.500000,2,2
